# SELECCIÓN DE MODELOS I

## 1. Importando las librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from random import shuffle
from mpl_toolkits.mplot3d import Axes3D
from linear_utils import LR

## 2. Analizando los datos de entrenamiento

### Cargando los datos de entrenamiento

Los datos corresponden a ventas en Ames, Iowa. Se obtuvieron de Kaggle: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview 

In [ ]:
# Indicamos la dirección y nombre del archivo
file_name_train = 'Data/train.csv'
file_name_test = 'Data/test.csv'
train_data = pd.read_csv(file_name_train)
test_data = pd.read_csv(file_name_test)

# Visualizamos el header de entrenamiento
train_data.head()

Nos interesa primero ver un modelo de regresión en 2 dimensiones por lo que se hará una prueba solo con el área del primer piso y el precio de la casa.

In [ ]:
# Se cargan datos de entrenamiento
x_pd = train_data['1stFlrSF']
y_pd = train_data['SalePrice']

x = x_pd.values.tolist()
y = y_pd.values.tolist()

x = np.array(x, dtype='float64')
x = x.reshape(x.shape[0], 1)

y = np.array(y, dtype='float64')
y = y.reshape(y.shape[0], 1)

# Se aleatoriza
idx = [i for i in range(len(x))]
shuffle(idx)
x = x[idx] 
y = y[idx]

# Se seccionan los datos de entrenamiento
split = 0.7
x_train = x[0:round(split*x.shape[0])]
x_test = x[round(split*x.shape[0]):]
y_train = y[0:round(split*y.shape[0])]
y_test = y[round(split*y.shape[0]):]

# Creamos nuestro objeto lr
lr = LR()

# Se normalizan los datos
x_norm = lr.norm(x_train)
y_norm = lr.norm(y_train)

x_train_norm = lr.norm(x_train)
y_train_norm = lr.norm(y_train)

x_test_norm = lr.norm(x_test)
y_test_norm = lr.norm(y_test)

print(x_train_norm.shape, x_test_norm.shape, y_train_norm.shape, y_test_norm.shape)


## 3. Sobreajuste vs subajuste

### 3.1 LMS

In [ ]:
# Añadimos la unidad de sesgo
x_train_norm_b =  np.insert(x_train_norm, 0, 1, axis=1)
x_train_norm_b = x_train_norm_b.T
x_test_norm_b =  np.insert(x_test_norm, 0, 1, axis=1)
x_test_norm_b = x_test_norm_b.T
x_norm_b =  np.insert(x_norm, 0, 1, axis=1)
x_norm_b = x_norm_b.T
x_train_b =  np.insert(x_train, 0, 1, axis=1)
x_train_b = x_train_b.T

# Optimizamos con descenso
num_iter = 600
alpha = 0.5
it = np.linspace(0 ,num_iter, num_iter)
w_LMS, j_LMS_train, j_LMS_test = lr.fit_gd(x_train_norm_b, y_train_norm, x_test_norm_b, y_test_norm, num_iter, alpha)

# Realizamos predicciones del entrenamiento
preds_LMS_norm_train = lr.h(w_LMS, x_train_norm_b)
preds_LMS_train = lr.denormalize(preds_LMS_norm_train, min(y_train), max(y_train))

# Realizamos predicciones en validación
preds_LMS_norm_test = lr.h(w_LMS, x_test_norm_b)
preds_LMS_test = lr.denormalize(preds_LMS_norm_test, min(y_test), max(y_test))

# Graficamos entrenamiento
plt.figure()
plt.scatter(x_train, y_train, alpha = 1)
plt.plot(x_train, preds_LMS_train.T, 'r', linewidth=3)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Graficamos validación
plt.figure()
plt.scatter(x_test, y_test, alpha = 1)
plt.plot(x_test, preds_LMS_test.T, 'g', linewidth=3)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Graficamos el costo
plt.figure()
plt.plot(it, j_LMS_train, 'r', linewidth=3, label='Costo entrenamiento')
plt.plot(it, j_LMS_test, 'g', linewidth=3, label='Costo validación')
plt.title('Costo cost(w) en el entrenamiento')
plt.xlabel('Costo cost(w)')
plt.ylabel('Iteraciones')
plt.legend()


### 3.2 NE

In [ ]:
# Optimizamos con NE
w_NE = lr.fit_ne(x_train_norm_b.T, y_train_norm)

# Realizamos predicciones del entrenamiento
preds_NE_norm_train = lr.h(w_NE, x_train_norm_b)
preds_NE_train = lr.denormalize(preds_NE_norm_train, min(y_train), max(y_train))

# Realizamos predicciones en validación
preds_NE_norm_test = lr.h(w_NE, x_test_norm_b)
preds_NE_test = lr.denormalize(preds_NE_norm_test, min(y_test), max(y_test))

# Graficamos entrenamiento
plt.figure()
plt.scatter(x_train, y_train, alpha = 1)
plt.plot(x_train, preds_NE_train.T, 'r', linewidth=3)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Graficamos validación
plt.figure()
plt.scatter(x_test, y_test, alpha = 1)
plt.plot(x_test, preds_NE_test.T, 'g', linewidth=3)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

## 4. Regresión lineal ponderada
Se sabe entonces que el error medio cuadrático se escribe de la siguiente forma:

$$J(w)=\frac{1}{2m}\sum_{i=1}^{m}\theta_{i}(h(x)^{(i)}-y^{(i)})^{2}$$

Se crea entonces la función de ponderación está dada por:

$$\theta_{i} = e^{-\frac{(x^{(i)}-x)^2}{2\tau^{2}}}$$

In [ ]:
# Función de ponderacion
def theta(x_i, x, tau=1):
    """
        A weighting bell function that computes the level of importance of a single training input x_i with respect to the specified pivot x.
    
        params:
            x_i: [np_array] an input vector with n feature variables with dimensions (nx1)
            x: [np_array] an input vector with n feature variables with dimensions (nx1), which we assigned as our pivot evaluation.
            tau: [double] a scalar representing the bandwith of the weighting function
                
        return:
            theta: [np_array] the final weight assigned to this particular fit
    """
    a = -(x_i - x).T.dot((x_i-x))/(2*tau**2)
    return np.exp(a)
    

In [ ]:
# Create two input vectors x_i and x
x_i = np.array([1, 2], dtype='float64')
x_1 = np.array([2, 5], dtype='float64')
tau = 2
w = theta(x_i, x_1, tau)
print(x_i.shape, x_1.shape, w)

### Creando la función de costo ponderada

In [ ]:
# Función de costo ponderada
def J_theta(w, x, y, x_pivot, tau):
    """
        params:
            w: [np_array] a vector of weights with dimensions (nx1), where n represents the number of weights.
            x: [np_array] a vector of feature variables with dimensions (nxm), 
                where n represents the number of feature variables and m the number of training examples
            y: [np_array] a vector of feature variables with dimensions (mx1), 
                where m represents the number of target variables
            x_pivot: [np_array] the pivot where the cost function will be computed of dimensions (nx1) 
            tau: [double] bandwith of weighting function
        returns:
            cost: [double] the mean squared error
    """
    # Get number o training examples 
    m = x.shape[1]
    
    # Initialize error term
    e = 0
    
    # Iterate over m training examples
    for i in range(m):
        
        # Get single coordinates
        x_i = np.array(x.T[i])
        y_i = y[i]
                    
        # Compute weighted error 
        th = theta(x_i.T, x_pivot, tau)
        e_i = th*(w.T.dot(x_i.T) - y_i)*(w.T.dot(x_i.T) - y_i)

        # Accumulate error
        e = e + e_i
    
    return (1/(2*x.shape[1]))*e

In [ ]:
# Vamos a probar la función de costo

# Generamos datos aleatoriamente
X = 2*np.random.rand(100,1)
Y = 4 + 3*X**2 + 5*X**3 + np.random.randn(100,1)

# Añadimos la unidad de sesgo
X_b =  np.insert(X, 0, 1, axis=1)
X_b = np.transpose(X_b)

# Inicializamos los pesos aleatoriamente (opcional)
w = np.array(np.random.randn(X_b.shape[0], 1))

# se calcula la hipótesis para un solo dato pivote
x_pivot = np.array([1, 1.5], dtype='float64')
tau = 0.01
mse = J_theta(w, X_b, Y, x_pivot, tau)
print(mse)

### Calculando el gradiente
Recordando que el gradiente está dado por:

$$\bigtriangledown _{w} cost = \frac{1}{m} \sum_{i=1}^{m} \theta^{(i)}(y^{(i)}-h(x^{(i)}))x^{(i)} $$

In [ ]:
# Gradiente de la función de costo
def dJ_theta(w, x, y, x_pivot, tau):
    """
        params:
            w: [np_array] a vector of weights with dimensions (nx1), where n represents the number of weights.
            x: [np_array] a vector of feature variables with dimensions (nxm), 
                where n represents the number of feature variables and m the number of training examples
            y: [np_array] a vector of feature variables with dimensions (mx1), 
                where m represents the number of target variables
        returns:
            dJ: [double] the derivative of the mean squared error
    """
    
    # Get number o training examples 
    m = x.shape[1]
    
    # Initialize error term
    e = 0
    
    # Iterate over m training examples
    for i in range(m):
        
        # Get single coordinates
        x_i = x.T[i]
        y_i = y[i]
        
        # Compute weighted error 
        th = theta(x_i, x_pivot, tau)
        e_i = (w.T.dot(x_i.T)-y_i)*th*x_i
                    
        # Accumulate error
        e = e + e_i
    
    return np.array([(1/(x.shape[1]))*e], dtype='float64')

In [ ]:
# Vamos a probar la función de gradiente

# Inicializamos los pesos aleatoriamente (opcional)
w = np.random.randn(X_b.shape[0], 1)

# se calcula la hipótesis para un solo dato pivote
x_pivot = np.array([1, 1.5], dtype='float64')

deriv_J = dJ_theta(w, X_b, Y, x_pivot, tau=2)
print(deriv_J.shape, w.shape)

### Calculando el descenso por gradiente para un solo punto pivote

$$ w:=w- \alpha \bigtriangledown _{w} cost $$


In [ ]:
def optimizar_LRP(x, y, num_iter, alpha, x_pivot, tau, w = None):
    """
    We calculate gradient descent for minimizing the weighted MSE to obtain the best linear hypothesis.
        params:
            x: [np_array] a vector of feature variables with dimensions (nxm), 
                where n represents the number of feature variables and m the number of training examples
            y: [np_array] a vector of feature variables with dimensions (mx1), 
                where m represents the number of target variables
            num_iter: [int] an integer indicating the number of iterations of the Gradient Descent algorithm
            alpha: [double] learning rate constant specifying the magnitude update step
            w: [np_array] vector that contains the initial weights to start optimzing the model with dimensions (n x 1)
                
        return:
            j: [np_array] a vector (num_iter x 1) containing all cost function evaluations during training
            w: [np_array] a vector of the final optimized weights with dimensions (nx1)
    """
    
    if w is None:
        # Inicializamos los pesos aleatoriamente
        w = np.random.randn(x.shape[0], 1)

    # se generan los vectores
    it = np.arange(0, num_iter)
    j = np.zeros(num_iter)
    
    # Se optimiza el modelo por el numero de iteraciones
    for i in range(num_iter):

        # Calculamos la hipótesis
        preds = w.T.dot(x)

        # Actualizamos los pesos
        w = w - alpha * dJ_theta(w, x, y, x_pivot, tau).T

        # Guardamos el costo
        j[i] = J_theta(w, x, y, x_pivot, tau)

    return w, j

### Ejemplo modelo simple: un pivote

In [ ]:
# Generamos datos aleatoriamente
X = 2*np.random.rand(100,1)
X = np.array(X, dtype='float64')
Y = 4 + 2*X + 3*X**2 + 5*X**3 + np.random.randn(100,1)
Y = np.array(Y, dtype='float64')

# Añadimos la unidad de sesgo
X_b =  np.insert(X, 0, 1, axis=1)
X_b = np.transpose(X_b)

# Inicializamos los pesos aleatoriamente (opcional)
w = np.random.randn(X.shape[0], 1)

# Se gráfican los datos
plt.figure()
plt.scatter(X, Y, alpha = 1)
plt.title('Random data')
plt.xlabel('X')
plt.ylabel('Y')

# Se corre el algoritmo para un solo dato pivote
x_pivot = np.array([1, 1], dtype='float64')
num_iter = 2500
alpha = 0.1
tau = 0.01
w, j = optimizar_LRP(X_b, Y, num_iter, alpha, x_pivot, tau)
print(w.shape)

# Graficamos la hipotesis lineal
preds = w.T.dot(X_b)
plt.plot(X, preds.T, 'r')

# Graficamos en el punto pivote
preds =  w.T.dot(x_pivot)
plt.scatter(x_pivot[1], preds, s=100)

# Graficamos el costo
it = np.linspace(0, num_iter, num_iter)
plt.figure()
plt.title('Gráfica del costo en el tiempo')
plt.xlabel('Iteraciones')
plt.ylabel('Costo') 
plt.plot(it, j)
plt.show()

print(j[-1])

### Ejemplo modelo simple: varios pivotes

In [ ]:
def predict_LRP(X_b, Y, num_iter, alpha, pivots, tau):
    
    """
        Fit the data in all pivots using the weighting linear regression cost function.     
    """
    
    # Initialize predictions vector
    preds = []
    
    # Iterate in pivots
    for pivot in pivots:
        print('Estimando en x = {0}'.format(pivot))
        x_pivot = np.array([1, pivot], dtype='float64')
        w, j = optimizar_LRP(X_b, Y, num_iter, alpha, x_pivot, tau)

        # Graficamos en el punto pivote
        pred = w.T.dot(x_pivot)
        preds.append(pred)
    return(preds)

In [ ]:
# Se corre el algoritmo para 10 datos pivotes
pivots = np.linspace(0, 2, 10, dtype='float64')
num_iter = 2500
alpha = 0.1
tau = 0.1

preds = predict_LRP(X_b, Y, num_iter, alpha, pivots, tau)

# Se gráfican los datos
plt.figure()
plt.scatter(X, Y, alpha = 1)
plt.title('Random data')
plt.xlabel('X')
plt.ylabel('Y')
 
# Graficamos la curva obtenida
plt.scatter(pivots, preds, s=100, c='r')
plt.plot(pivots, preds, 'r')

### Ejemplo modelo simple: variando el ancho de banda

In [ ]:
# Se gráfican los datos
plt.figure()
plt.scatter(X, Y, alpha = 1)
plt.title('Random data')
plt.xlabel('X')
plt.ylabel('Y')

# Se corre el algoritmo para 5 datos pivotes
pivots = np.linspace(0, 2, 5, dtype='float64')
num_iter = 1500
alpha = 0.1
tau = 0.01

# Vector de taus
taus = np.linspace(0.1, 1, 5, dtype='float64')

for tau in taus:
    print('TAU {0}'.format(tau))
    preds = predict_LRP(X_b, Y, num_iter, alpha, pivots, tau)
    plt.plot(pivots, preds, label=tau)
    print('\n')
    
plt.legend(loc='lower right')

### Ejemplo casas: predecir un pivote

In [ ]:
# Graficamos los datos
plt.figure()
plt.scatter(x_train, y_train, alpha = 1)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Calculando el pivote
x_pivot = np.array([1, 1000], dtype='float64') 
x_pivot_norm = (x_pivot - min(x_train))/(max(x_train) - min(x_train))

# Se corre el algoritmo para un solo dato pivote
num_iter = 1000
alpha = 0.1
tau = 0.1
w, j = optimizar_LRP(x_train_norm_b, y_train_norm, num_iter, alpha, x_pivot_norm, tau)

# Graficamos en el punto pivote denormalizado
preds_norm = w.T.dot(x_pivot_norm)
preds = lr.denormalize(preds_norm, min(y_train), max(y_train))
plt.scatter(x_pivot[1], preds, s=100)

# Graficamos el costo
it = np.linspace(0, num_iter, num_iter)
plt.figure()
plt.title('Gráfica del costo en el tiempo')
plt.xlabel('Iteraciones')
plt.ylabel('Costo') 
plt.plot(it, j)
plt.show()

### Ejemplo casas: predecir varios pivotes datos de entrenamiento

In [ ]:
# Graficamos los datos
plt.figure()
plt.scatter(x_train, y_train, alpha = 1)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# se calculan los pivotes
pivots = np.linspace(500, 2000, 10, dtype='float64') 
pivots_norm = (pivots - min(x_train))/(max(x_train) - min(x_train))

# Se corre el algoritmo para 10 datos pivotes
num_iter = 1000
alpha = 0.1
tau = 0.1

# Vector de taus
preds_norm = predict_LRP(x_train_norm_b, y_train_norm, num_iter, alpha, pivots_norm, tau)
preds = lr.denormalize(preds_norm, min(y_train), max(y_train))

# Se grafican las estimaciones
plt.scatter(pivots, preds, s=100, c='r')
plt.plot(pivots, preds, 'r')

### Ejemplo casas: predecir varios pivotes datos de validación

In [ ]:
# Graficamos los datos de validación
plt.figure()
plt.scatter(x_test, y_test, alpha = 1)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Se grafican las estimaciones
plt.scatter(pivots, preds, s=100, c='y')
plt.plot(pivots, preds, 'y')

### Ejemplo casas: sin normalizar

In [ ]:
# Graficamos los datos
plt.figure()
plt.scatter(x_train, y_train, alpha = 1)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Se corre el algoritmo para 10 datos pivotes
pivots = np.linspace(500, 2000, 10, dtype='float64')
num_iter = 800
alpha = 0.1
tau = 0.1

# Vector de taus
preds = predict_LRP(x_train_b, y_train, num_iter, alpha, pivots, tau)

# Se grafican las estimaciones
plt.scatter(pivots, preds, s=100, c='r')
plt.plot(pivots, preds, 'r')

### Ejemplo casas: variando el ancho de banda

In [ ]:
# Se gráfican los datos
plt.figure()
plt.scatter(x_train, y_train, alpha = 1)
plt.title('Casas en Iowa')
plt.xlabel('Área del primer piso [m2]')
plt.ylabel('Precio de venta [$]')

# Se corre el algoritmo para 5 datos pivotes
pivots = np.linspace(500, 2000, 10, dtype='float64') 
pivots_norm = (pivots - min(x_train))/(max(x_train) - min(x_train))
num_iter = 800
alpha = 0.1

# Vector de taus
taus = np.linspace(0.1, 1, 5, dtype='float64')

for tau in taus:
    print('TAU {0}'.format(tau))
    preds_norm = predict_LRP(x_train_norm_b, y_train_norm, num_iter, alpha, pivots_norm, tau)
    preds = lr.denormalize(preds_norm, min(y_train), max(y_train))
    plt.plot(pivots, preds, label=tau)
    print('\n')
    
plt.legend(loc='lower right')

## 5. Regularización

### Regularización L2: Descenso por Gradiente

La función de costo $cost(w)$ con el término de regularización L2 está dada por:

$$cost(w)=(\vec{w}^{T}\phi(X)-\vec{y})^{T}(\vec{w}^{T}\phi(X)-\vec{y})+ \frac{\lambda}{2} \vec{w}^{T}{\vec{w}}$$

Se puede calcular el gradiente de la función, lo que se define como:

$$\nabla_{w} cost(w) = \frac{1}{m} X^{T}(X \vec{w}-\vec{y}) + \lambda\vec{w}$$

### Regularización L2: Ecuaciones Normales

Igualando el gradiente $\nabla _{w} cost(w)$ a cero y despejando para $\vec{w}$ se obtienen las siguientes ecuaciones:

$$\vec{w} = (X^{T}X + \lambda I)^{-1} X^{T}y$$


Programamos la función de gradiente
$$\nabla_{w} cost(w) = \frac{1}{m} X^{T}(X \vec{w}-\vec{y}) + \lambda\vec{w}$$

In [ ]:
    def dJ(w, phi, y, l2 = 0):
        """
            params:
                w: [np_array] a vector of weights with dimensions (nx1), where n represents the number of weights.
                x: [np_array] a vector of feature variables with dimensions (nxm),
                    where n represents the number of feature variables and m the number of training examples
                y: [np_array] a vector of feature variables with dimensions (mx1),
                    where m represents the number of target variables
                l2: [double] L2 regularization constant 
                
            returns:
                dJ: [double] the derivative of the mean squared error
        """
        e = y.T - w.T.dot(phi.T)
        return (1 / (phi.shape[0])) * phi.T.dot(e.T) + l2*w

Función de costo:

$$cost(w)=(\vec{w}^{T}\phi(X)-\vec{y})^{T}(\vec{w}^{T}\phi(X)-\vec{y})+ \frac{\lambda}{2} \vec{w}^{T}{\vec{w}}$$

In [ ]:
def cost(w, phi, y, l2=0):
    """
        params:
            w: [np_array] a vector of weights with dimensions (nx1), where n represents the number of weights.
            x: [np_array] a vector of feature variables with dimensions (nxm),
                where n represents the number of feature variables and m the number of training examples
            y: [np_array] a vector of feature variables with dimensions (mx1),
                where m represents the number of target variables
        returns:
            cost: [double] the mean squared error
    """
    e = y.T - w.T.dot(phi.T)
    return (1/(2*phi.shape[1])) * (np.sum(np.square(e))) + (l2/2)*w.T.dot(w)

Función de optimización

In [ ]:
    def optimizar_LMS(x_train, y_train, x_test, y_test, num_iter, alpha, w=None, l2=0):
        """
        We calculate gradient descent for minimizing the MSE to obtain the best linear hypothesis.
            params:
                x: [np_array] a vector of feature variables with dimensions (nxm),
                    where n represents the number of feature variables and m the number of training examples
                y: [np_array] a vector of feature variables with dimensions (mx1),
                    where m represents the number of target variables
                num_iter: [int] an integer indicating the number of iterations of the Gradient Descent algorithm
                alpha: [double] learning rate constant specifying the magnitude update step
                w: [np_array] vector that contains the initial weights to start optimzing the model with dimensions (n x 1)
                l2: [double] L2 regularization constant 
                
            return:
                j: [np_array] a vector (num_iter x 1) containing all cost function evaluations during training
                w: [np_array] a vector of the final optimized weights with dimensions (nx1)
        """

        if w is None:
            # Inicializamos los pesos aleatoriamente
            w = np.random.randn(x_train.shape[0], 1)

        # se generan los vectores
        j_train = np.zeros(num_iter)
        j_test = np.zeros(num_iter)

        # Se optimiza el modelo por el numero de iteraciones
        for i in range(num_iter):

            # Actualizamos los pesos
            w = w + alpha * dJ(w, x_train, y_train, l2=l2)

            # Guardamos el costo
            j_train[i] = cost(w, x_train, y_train, l2=l2)

            # Guardamos el costo
            j_test[i] = cost(w, x_test, y_test, l2=l2)

        return w, j_train, j_test

In [ ]:
%matplotlib notebook

# Generamos datos aleatoriamente
m = 50
noise = np.array([1*np.random.uniform(size=m)]).T

X = np.array([2*np.linspace(-20, 20, m)]).T
Y = -40 - 3*X - 5*X**2 + 10*X**3 + 2*X**7 + noise

# Aleatorizamos
# Se aleatoriza
idx = [i for i in range(len(X))]
shuffle(idx)
X = X[idx] 
Y = Y[idx]

# Normalizamos
X_norm = lr.norm(X)
Y_norm = lr.norm(Y)

# Dividimos
split = 0.7
x_train_norm = X_norm[0:round(split*X_norm.shape[0])]
x_test_norm = X_norm[round(split*X_norm.shape[0]):]
y_train_norm = Y_norm[0:round(split*Y_norm.shape[0])]
y_test_norm = Y_norm[round(split*Y_norm.shape[0]):]

x_train = X[0:round(split*X.shape[0])]
x_test = X[round(split*X.shape[0]):]
y_train = Y[0:round(split*Y.shape[0])]
y_test = Y[round(split*Y.shape[0]):]

# Se corre el algoritmo para K lambdas
num_lambdas = 4
lambdas = np.linspace(0, 0.5, num_lambdas, dtype='float64') 
num_iter = 300
alpha = 0.01
it = np.linspace(0 ,num_iter, num_iter)
degree = 10

# Basis functions
degrees = range(degree + 1)
phi_train = lr.expand(x_train_norm, bf=lr.polynomial_basis_function, bf_args=degrees[1:])
phi_test = lr.expand(x_test_norm, bf=lr.polynomial_basis_function, bf_args=degrees[1:])
w = np.random.randn(phi_train.shape[1], 1)

# Graficamos 
fig, axs = plt.subplots(num_lambdas, 3, figsize=(10, 10))
fig.subplots_adjust(hspace=1, wspace=1)

for i, l2 in enumerate(lambdas):

    w_LMS, j_LMS_train, j_LMS_test = optimizar_LMS(phi_train, y_train_norm, phi_test, y_test_norm, num_iter, alpha, w=w, l2=l2)

    # Realizamos predicciones del entrenamiento
    preds_train = w_LMS.T.dot(phi_train.T)
    preds_train = lr.denormalize(preds_train, min(Y), max(Y))
    preds_test = w_LMS.T.dot(phi_test.T)
    preds_test = lr.denormalize(preds_test, min(Y), max(Y))

    # Realizamos predicciones en validación
    axs[i][0].scatter(x_train, y_train, alpha = 1)
    axs[i][0].scatter(x_train, preds_train.T, label='Lambda = {0:0.03f}'.format(l2))
    axs[i][0].set_title('Casas en Iowa')
    axs[i][0].set_xlabel('Área del primer piso [m2]')
    axs[i][0].set_ylabel('Precio de venta [$]')
    axs[i][0].legend()
    
    axs[i][1].scatter(x_test, y_test, alpha = 1)
    axs[i][1].scatter(x_test, preds_test.T, label='Lambda = {0:0.03f}'.format(l2))
    axs[i][1].set_title('Casas en Iowa')
    axs[i][1].set_xlabel('Área del primer piso [m2]')
    axs[i][1].set_ylabel('Precio de venta [$]')
    axs[i][1].legend()

    # Graficamos el costo
    axs[i][2].plot(it, j_LMS_train, 'r', linewidth=3, label='Costo entrenamiento {0:0.03f}'.format(j_LMS_train[-1]))
    axs[i][2].plot(it, j_LMS_test, 'g', linewidth=3, label='Costo validación {0:0.03f}'.format(j_LMS_test[-1]))
    axs[i][2].set_title('Costo cost(w) en el entrenamiento')
    axs[i][2].set_xlabel('Costo cost(w)')
    axs[i][2].set_ylabel('Iteraciones')
    axs[i][2].legend()
    

Ecuaciones normales

$$\vec{w} =(\lambda I + \phi^{T} \phi)^{-1} \phi ^{T} \vec{y}$$

In [ ]:
def fit_ne(phi, y, l2=0):
    """
        params:
            x: [np_array] a vector of feature variables with dimensions (mxn), 
                where n represents the number of feature variables and m the number of training examples
            y: [np_array] a vector of feature variables with dimensions (nx1), 
                where m represents the number of target variables
                
        return:
            w: [np_array] a vector of the final optimized weights with dimensions (nx1)
    """
    return np.linalg.inv(l2*np.identity(phi.shape[1]) + phi.T.dot(phi)).dot(phi.T).dot(y) 

In [ ]:
# Se corre el algoritmo para K lambdas
num_lambdas = 5
lambdas = np.linspace(0, 0.4, num_lambdas, dtype='float64') 
it = np.linspace(0 ,num_iter, num_iter)
degree = 7

# Basis functions
degrees = range(degree + 1)
phi_train = lr.expand(x_train_norm, bf=lr.polynomial_basis_function, bf_args=degrees[1:])
phi_test = lr.expand(x_test_norm, bf=lr.polynomial_basis_function, bf_args=degrees[1:])
w = np.random.randn(phi_train.shape[1], 1)

# Graficamos 
fig, axs = plt.subplots(num_lambdas, 2, figsize=(10, 10))
fig.subplots_adjust(hspace=1, wspace=1)

for i, l2 in enumerate(lambdas):

    w_LMS = fit_ne(phi_train, y_train_norm, l2=l2)

    # Realizamos predicciones del entrenamiento
    preds_train = w_LMS.T.dot(phi_train.T)
    preds_train = lr.denormalize(preds_train, min(Y), max(Y))
    preds_test = w_LMS.T.dot(phi_test.T)
    preds_test = lr.denormalize(preds_test, min(Y), max(Y))
    
    # Calculamos el costo
    j_train = cost(w, phi_train, y_train_norm, l2=l2)
    j_test = cost(w, phi_test, y_test_norm, l2=l2)

    # Realizamos predicciones en validación
    axs[i][0].scatter(x_train, y_train, alpha = 1)
    axs[i][0].scatter(x_train, preds_train.T, label='Lambda = {0:0.03f}'.format(l2))
    axs[i][0].set_title('Casas en Iowa - {0:0.03}'.format(j_train[0][0]))
    axs[i][0].set_xlabel('Área del primer piso [m2]')
    axs[i][0].set_ylabel('Precio de venta [$]')
    axs[i][0].legend()
    
    axs[i][1].scatter(x_test, y_test, alpha = 1)
    axs[i][1].scatter(x_test, preds_test.T, label='Lambda = {0:0.03f}'.format(l2))
    axs[i][1].set_title('Casas en Iowa - {0:0.03}'.format(j_test[0][0]))
    axs[i][1].set_xlabel('Área del primer piso [m2]')
    axs[i][1].set_ylabel('Precio de venta [$]')
    axs[i][1].legend()